# Implementing a Convolutional Neural Network and training it with the MNIST handwriting dataset.

I will try to implement this using PyTorch, and if not, I will find some way to implement the CNN.

### Libraries needed

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import optim
from torch import nn